In [1]:
!pip install langgraph langchain langchain-openai python-dotenv

In [2]:
import os
from dotenv import load_dotenv
from typing import TypedDict, Annotated, Sequence, Literal
import operator
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode # Para facilitar la creación de nodos de herramientas
from langchain_core.tools import tool # Para crear herramientas fácilmente

In [3]:
load_dotenv()

True

In [7]:
from langchain_ollama.chat_models import ChatOllama
llm = ChatOllama(model="llama3.2:latest")

In [8]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [9]:
@tool
def magic_search_tool(query: str) -> str:
    """Simula una búsqueda mágica. Si la consulta es sobre 'LangGraph', devuelve información específica.
    Para otras consultas, indica que no encontró nada específico."""
    print(f"---> Herramienta 'magic_search_tool' llamada con la consulta: {query}")
    if "langgraph" in query.lower():
        return "LangGraph es una biblioteca para construir aplicaciones LLM con estado y ciclos. Es útil para agentes complejos."
    return f"No se encontró información específica para '{query}' en la búsqueda mágica."

tools = [magic_search_tool]

# Langchain y LangGraph pueden trabajar con herramientas de forma más estructurada.
# Aquí, vincularemos las herramientas al LLM para que sepa cuándo y cómo llamarlas.
llm_with_tools = llm.bind_tools(tools)

print("Herramientas definidas y vinculadas al LLM.")

Herramientas definidas y vinculadas al LLM.


In [10]:
def call_model(state: AgentState):
    """Llama al LLM con el historial de mensajes actual. 
    El LLM puede responder o solicitar una llamada a herramienta."""
    print("---NODO: call_model---")
    messages = state["messages"]
    response = llm_with_tools.invoke(messages)
    # Devolvemos la respuesta del LLM para añadirla al estado
    return {"messages": [response]}

tool_node = ToolNode(tools)

print("Nodos definidos.")

Nodos definidos.


In [11]:
def should_continue(state: AgentState) -> Literal["action", "end"]:
    """Decide la siguiente acción basada en la última respuesta del LLM."""
    print("---NODO CONDICIONAL: should_continue---")
    last_message = state["messages"][-1]
    # Si el LLM hizo una llamada a herramienta, entonces vamos al nodo de acción (tool_node)
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        print("Decisión: LLM solicitó una herramienta. Ir a 'action'.")
        return "action"
    # Si no hay llamada a herramienta, el LLM ha respondido, entonces terminamos.
    print("Decisión: LLM respondió. Ir a 'end'.")
    return "end"

print("Lógica condicional definida.")

Lógica condicional definida.


In [12]:
# Crear el grafo de estado
workflow = StateGraph(AgentState)

# Añadir los nodos al grafo
workflow.add_node("agent", call_model) # Nodo donde el agente (LLM) decide
workflow.add_node("action", tool_node)  # Nodo que ejecuta la acción/herramienta

# Definir el punto de entrada del grafo
workflow.set_entry_point("agent")

# Añadir las aristas condicionales
workflow.add_conditional_edges(
    "agent", # Nodo de origen
    should_continue, # Función que decide la ruta
    {
        "action": "action", # Si should_continue devuelve "action", ir al nodo "action"
        "end": END          # Si should_continue devuelve "end", terminar el grafo
    }
)

# Añadir una arista desde el nodo de acción de vuelta al agente
# Después de ejecutar una herramienta, queremos que el agente procese el resultado de la herramienta.
workflow.add_edge("action", "agent")

# Compilar el grafo en una aplicación ejecutable
app = workflow.compile()
print("Grafo compilado y listo.")

Grafo compilado y listo.


In [13]:
inputs1 = {"messages": [HumanMessage(content="Hola, ¿cómo estás?")]}
try:
    print("\n--- Ejecución 1: Saludo Simple ---")
    for event in app.stream(inputs1):
        for key, value in event.items():
            print(f"Evento del grafo: Nodo='{key}', Estado actualizado='{value}'")
        print("---")
    # La respuesta final estará en el último estado del nodo 'agent' o 'end'
    final_state_1 = app.invoke(inputs1)
    print(f"Respuesta final (Saludo): {final_state_1['messages'][-1].content}")
except Exception as e:
    print(f"Error en la ejecución 1: {e}")


# Ejecutar con una pregunta que podría requerir la herramienta
inputs2 = {"messages": [HumanMessage(content="¿Qué sabes sobre LangGraph?")]}
try:
    print("\n--- Ejecución 2: Pregunta sobre LangGraph (debería usar herramienta) ---")
    for event in app.stream(inputs2, {"recursion_limit": 5}): # Añadir límite de recursión
        for key, value in event.items():
            print(f"Evento del grafo: Nodo='{key}', Estado actualizado='{value}'")
        print("---")
    final_state_2 = app.invoke(inputs2, {"recursion_limit": 5})
    print(f"Respuesta final (LangGraph): {final_state_2['messages'][-1].content}")
except Exception as e:
    print(f"Error en la ejecución 2: {e}")


# Ejecutar con una pregunta que la herramienta no cubre
inputs3 = {"messages": [HumanMessage(content="¿Cuál es la capital de Francia?")]}
try:
    print("\n--- Ejecución 3: Pregunta general (no debería usar herramienta específica) ---")
    for event in app.stream(inputs3, {"recursion_limit": 5}):
        for key, value in event.items():
            print(f"Evento del grafo: Nodo='{key}', Estado actualizado='{value}'")
        print("---")
    final_state_3 = app.invoke(inputs3, {"recursion_limit": 5})
    print(f"Respuesta final (Capital): {final_state_3['messages'][-1].content}")
except Exception as e:
    print(f"Error en la ejecución 3: {e}")


--- Ejecución 1: Saludo Simple ---
---NODO: call_model---
---NODO CONDICIONAL: should_continue---
Decisión: LLM solicitó una herramienta. Ir a 'action'.
Evento del grafo: Nodo='agent', Estado actualizado='{'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-05-16T17:34:57.240672Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6682104459, 'load_duration': 5542370209, 'prompt_eval_count': 192, 'prompt_eval_duration': 691344417, 'eval_count': 17, 'eval_duration': 447762125, 'model_name': 'llama3.2:latest'}, id='run--5a87c034-4604-473d-b509-bd05d9635da8-0', tool_calls=[{'name': 'magic_search_tool', 'args': {'query': ''}, 'id': 'd8d7766a-91bf-486c-becc-141714b0ee65', 'type': 'tool_call'}], usage_metadata={'input_tokens': 192, 'output_tokens': 17, 'total_tokens': 209})]}'
---
---> Herramienta 'magic_search_tool' llamada con la consulta: 
Evento del grafo: Nodo='action', Estado actualizado='{'messages': [ToolMe